In [75]:
import re
from time import sleep
from openai import OpenAI
from os import getenv
import pandas as pd
import numpy as np

from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

In [76]:
openai_client = OpenAI(api_key=getenv("OPENAI_API_KEY"))

In [91]:
sample = [
  { "role": "user", "content": "Site: https://mercadolivre.com.br\nSite_info: Site de compras\nQuery: Quero mouse de até 300 reais"},
      { "role": "assistant", "content": "Estou no mercado livre\npreciso buscar por mouses\ndevo achar um input\n-####-\nextract_elements('input', 'True')"},
      { "role": "user", "content": "Extracted elements:\nElement: input Name: as_word Placeholder: Buscar produtos, marcas e muito mais…"},
      { "role": "assistant", "content": "Encontrei o input\npreciso buscar por mouses\ndevo digitar no input\n####\interact_with_element('input', 'fill', 'mouse')"},
      { "role": "user", "content": "Texto 'mouse' inserido no elemento 'input'."},
      { "role": "assistant", "content": "Digitei minha busca\npreciso iniciar a busca\ndevo buscar um botao submit\n-####-\nextract_elements('button[type=\"submit\"]', 'True')"},
      { "role": "user", "content": "Extracted elements:\nElement: button Classes: nav-search-btn\nElement: button Classes: ui-search-price-filter-action-btn"},
      { "role": "assistant", "content": "Encontrei dois botoes\num tem a classe search\ndevo clicar nele\n####\interact_with_element('button', 'click')"},
      { "role": "user", "content": "Clique realizado no elemento 'button'"},
      { "role": "assistant", "content": "Agora preciso encontrar os mouses\ndevem estar em um card\ndevo buscar pela classe card\n-####-\nextract_elements('[class*=\"card\"]', 'True')"},
      { "role": "user", "content": "Extracted elements:\nElement: div Classes: andes-card brand-wrapper-desktop-new andes-card--f... Text: PatrocinadoPrecisão milimétrica, velocidade fatal ...\nElement: div Classes: andes-card__content brand-wrapper-desktop-new__con... Text: PatrocinadoPrecisão milimétrica, velocidade fatal ...\nElement: div Classes: andes-card poly-card poly-card--grid-card andes-ca... Text: LOGITECHMouse sem fio Logitech M280 com Pilha Incl...\nElement: div Classes: poly-card__portada Text: \nElement: div Classes: poly-card__content Text: LOGITECHMouse sem fio Logitech M280 com Pilha Incl...\nElement: div Classes: poly-card__footer Text: Patrocinado\nElement: div Classes: andes-card poly-card poly-card--grid-card andes-ca... Text: LOGITECHMouse sem fio Logitech M280 com Pilha Incl..."},
      { "role": "assistant", "content": "Varios elementos extraidos\nQuero o conteudo de cada produto\nA classe poly-card__content parece adequada\n-####-\nextract_elements('[class*=\"poly-card__content\"]', 'False')"},
      { "role": "user", "content": "Extracted elements:\nElement: div Clases: poly-card__content Text: LOGITECHMouse sem fio Logitech M280 com Pilha Inclusa - PretoPor Logitech Avaliação 4,9 de 5. (11,252 avaliações)4.9    (11252)R$99,90R$90,429% OFFem 12x R$8,86Frete grátisDisponível em 3 coresEnviado pelo \nElement: div Clases: poly-card__content Text: LOGITECHMouse sem fio Logitech M280 com Pilha Inclusa - CinzaPor Logitech Avaliação 4,9 de 5. (11,252 avaliações)4.9    (11252)R$98,66em 12x R$9,67Frete grátisDisponível em 4 coresEnviado pelo \nElement: div Clases: poly-card__content Text: LOGITECHMouse Sem Fio Logitech Signature M650 L- GrafitePor Logitech Avaliação 4,9 de 5. (373 avaliações)4.9    (373)R$219,90R$181,0417% OFFem 12x R$17,75Frete grátisEnviado pelo \nElement: div Clases: poly-card__content Text: Mouse Sem Fio Recarregável Wireles Optico Ergonômico BluetoothAvaliação 4,7 de 5. (2,944 avaliações)4.7    (2944)R$29em 12x R$2,84Chegará amanhãEnviado pelo \nElement: div Clases: poly-card__content Text: MAIS VENDIDOMouse Sem Fio Recarregável Wireles Optico Led Rgb ErgonômicoAvaliação 4,7 de 5. (25,756 avaliações)4.7    (25756)R$29,67em 12x R$2,91Chegará amanhãDisponível em 5 coresEnviado pelo \n"},
      { "role": "assistant", "content": """
Já tenho o que preciso
vou finalizar a sessão
vou formatar minha resposta
-####-
end()
### Produtos encontrados:

1️⃣ **Mouse Logitech M280 - Preto** 🖱️  
⭐ 4.9 (11.252) | ~~R$99,90~~ ➝ **R$90,42** (9% OFF) | 12x R$8,86 | 🚚 Grátis | 🎨 3 cores  

2️⃣ **Mouse Logitech M280 - Cinza**  
⭐ 4.9 (11.252) | **R$98,66** | 12x R$9,67 | 🚚 Grátis | 🎨 4 cores  

3️⃣ **Mouse Logitech Signature M650 L - Grafite**  
⭐ 4.9 (373) | ~~R$219,90~~ ➝ **R$181,04** (17% OFF) | 12x R$17,75 | 🚚 Grátis  

4️⃣ **Mouse Wireless Óptico Ergonômico Bluetooth**  
⭐ 4.7 (2.944) | **R$29,00** | 12x R$2,84 | 📦 Amanhã  

5️⃣ **🔥 MAIS VENDIDO - Mouse Wireless LED RGB**  
⭐ 4.7 (25.756) | **R$29,67** | 12x R$2,91 | 📦 Amanhã | 🎨 5 cores  
       """
      },
]

In [ ]:
prompt = """
  Você um assitente web scrapping. Com base na query do usuário e na página que 
  você se encontra, você deve extrair informações.

  Você vai receber: 
    - a query do usuário e informações do site em que se encontra
  Com base nisso você deve:
    - interagir com a página atrás das informações solicitadas
      - Você pode receber uma ecommerce para pesquisar produtos e retorna-los
      - Você pode rebeber um artigo comparativo de produtos e retorna-los
      - Você pode ter que entrar na página de um produto específico para extrair informações dele
      - Você pode estar no google e ter que entrar em um site atrás de um produto

  Ferramentas:
    - Para extrair dados da página use a função extract_elements(selector:str, truc: str, limit='10'), passando 
    o 'selector' para buscar o elemento desejado na página e o 'trunc' como 'True' caso queira que o texto seja truncado a no max 50 caracters + reticencias, e 'False' caso o contrario
      Ex.:
        Chamada: 
          extract_elements('button', 'True', '2')
        Resposta:
          Extracted elements:
          - Element: button Classes: btn-search Text: Search
          - Element: button Classes: btn-login Text: Login
    - Para interagir com a página use a função interact_with_element(selector: str, interaction: str, text: str)
      Ex.:
        Chamada: 
          interact_with_element('button.btn-search', 'click')
        Resposta:
          Clique realizado no elemento 'button.btn-search'
    - Para finalizar a sessão use a função end(), pule uma linha e dê a resposta para query do usuário
      Ex.:
        Chamada:
          end()
          A página contém as seguinte informações sobre o tópico solicitado:
          <...>

  Pontos de atenção
    - para seletores de atributos use aspas duplas e escape as aspas com \\
      Ex.:
        extract_elements('button[type=\"submit\"]', 'True', '2')

  Pense passo a passo, mas mantenha um rascunho mínimo para cada passo de pensamento, 
  com no máximo 5 palavras. Retorne a resposta no fim do texto, após o separador -####-
"""

def initial_prompt(url: str, site_info: str, query: str):
  return f"""
    Site: {url}
    Site_info: {site_info}
    Query: {query}
  """


def gpt(msgs):  
  response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=msgs,
    temperature=0.5,
    max_tokens=2000
  )

  return response.choices[0].message.content.strip()

def extract_func_and_args(text: str):
  pattern = r"([a-z_]+)\(('.*',?)*\)"
  match = re.search(pattern, text.strip())

  if match is None:
    raise Exception("No function match")

  name = match.group(1).strip()
  args = [arg.strip() for arg in match.group(2).strip().replace("'", "").split(",")] if match.group(2) else []

  return name, args

async def extract_data(url:str, site_info:str, query:str):
  result = ""
  async with async_playwright() as p:
    browser = await p.chromium.launch(headless=False)
    page = await browser.new_page()

    await page.goto(url)
    await page.wait_for_load_state()

    async def extract_elements(el_selector: str, trunc:bool, limit: int):
      elements = await page.query_selector_all(el_selector)
      formatted_elements = []
      
      for i, el in enumerate(elements):
        tag_name = (await el.evaluate('el => el.tagName')).lower()
        element = f"Element: {tag_name}"
        class_name = (await el.evaluate('el => el.className')).strip()
        text = (await el.text_content()).strip()

        if class_name:
          element += f" Classes: {class_name}"

        if text:
          formatted_text = text if not trunc else text[:50] + "..." if len(text) > 50 else text
          element += f" Text: {formatted_text}"

        if tag_name == "a":
          element += f" Href: {await el.get_attribute('href')}"

        if tag_name == "input":
          placeholder = await el.get_attribute("placeholder") or "sem placeholder"
          element += f" Name: {await el.get_attribute('name')} Placeholder: {placeholder}"
          
        formatted_elements.append(element)

        if len(formatted_elements) >= limit:
          break
      
      return "Extracted elements:\n" + "\n- ".join(formatted_elements)
  
    async def interact_with_element(el_selector: str, interaction: str, text: str):
      element = await page.query_selector(el_selector)
      
      if element is None:
        return f"Elemento com selector '{el_selector}' não encontrado"

      if interaction == "click":
        await element.click()
        return f"Clique realizado no elemento '{el_selector}'"

      if interaction == "fill":
        await element.type(text)
        return f"Texto '{text}' inserido no elemento '{el_selector}'."
      
      return "Interação não suportada."

    user_prompt = initial_prompt(url, site_info, query)
    print(f"[👤] {user_prompt}")

    chat = [
      { "role": "developer", "content": prompt },
      *sample,
      { "role": "user", "content": user_prompt },
    ]

    end = False
    while not end:
      sleep(3)
      full_response = gpt(chat)
      chat.append({ "role": "assistant", "content": full_response })
      print(f"[🤖] {full_response}")
      response = full_response.split("-####-")[1]
      func, args = extract_func_and_args(response)
      print(f"[🔍] func: {func} args: {args}")

      if func == "extract_elements":
        extracted = await extract_elements(
          args[0], 
          trunc=False if args[1] == 'False' else True,
          limit=int(args[2]) if len(args) == 3 else 10
        )
        print(f"[🔍] extracted: {extracted}")
        chat.append({ "role": "user", "content": extracted })
        continue

      if func == "interact_with_element":
        extracted = await interact_with_element(args[0], args[1], args[2] if len(args) == 3 else "")
        print(f"[🔍] extracted: {extracted}")
        chat.append({ "role": "user", "content": extracted })
        continue

      if func == "end":
        end = True
        result = response.replace("end()\n", "")


    await browser.close()

  return result

In [92]:
data = await extract_data(
  "https://mercadolivre.com.br",
  "Site de compras",
  "Quero uma lampada inteligente"
)
data

[👤] 
    Site: https://mercadolivre.com.br
    Site_info: Site de compras
    Query: Quero uma lampada inteligente
  
[🤖] Estou no mercado livre
preciso buscar por lâmpadas
devo achar um input
-####-
extract_elements('input', 'True')
[🔍] func: extract_elements args: ['input', 'True']
[🔍] extracted: Extracted elements:
Element: input Classes: nav-search-input Name: as_word Placeholder: Buscar produtos, marcas e muito mais…
[🤖] Encontrei o input
preciso buscar por lâmpadas
devo digitar no input
-####-
interact_with_element('input', 'fill', 'lâmpada inteligente')
[🔍] func: interact_with_element args: ['input', 'fill', 'lâmpada inteligente']
[🔍] extracted: Texto 'lâmpada inteligente' inserido no elemento 'input'.
[🤖] Digitei minha busca
preciso iniciar a busca
devo buscar um botão submit
-####-
extract_elements('button[type="submit"]', 'True')
[🔍] func: extract_elements args: ['button[type="submit"]', 'True']
[🔍] extracted: Extracted elements:
Element: button Classes: nav-search-btn
[🤖] En

'\n### Produtos encontrados:\n\n1️⃣ **Lâmpada Smart Bulbo Led Wi-fi - Avant**  \n⭐ 4.7 (2.925) | ~~R$49,00~~ ➝ **R$24,99** (49% OFF) | 🚚 Enviado pelo  \n\n2️⃣ **Lâmpada Led Wi-fi Smart - INTELBRAS**  \n⭐ 4.8 (640) | ~~R$70,90~~ ➝ **R$50,88** (28% OFF no Pix) | 💳 R$56,53 em outros meios | 🚚 Enviado pelo  \n\n3️⃣ **Lâmpada Led 10w Rgb Inteligente Wifi Smart - Elgin**  \n⭐ 4.8 (3.265) | **R$34,30** | 12x R$3,36 | 🚚 Enviado pelo  \n\n4️⃣ **Lâmpada Led com Wi-Fi - Avant**  \n⭐ 4.7 (2.925) | ~~R$158,90~~ ➝ **R$104,08** (34% OFF no Pix) | 🚚 Grátis | 💳 3x R$38,55 sem juros | 🚚 Enviado pelo  \n\n5️⃣ **Lampada Led Bulbo Alta Potencia 50w - MAIS VENDIDO**  \n⭐ 4.8 (3.967) | **R$21,90** | 🚚 Enviado pelo  \n\n6️⃣ **Lâmpada WI-FI INTELIGENTE - TP-LINK**  \n⭐ 4.7 (250) | ~~R$43,26~~ ➝ **R$36,99** (14% OFF) | 12x R$3,62 | 🚚 Enviado pelo  \n\n7️⃣ **Lâmpada Led Wifi Inteligente - NOVADIGITAL**  \n⭐ 4.7 (445) | ~~R$58,49~~ ➝ **R$50,31** (13% OFF) | 12x R$4,93 | 🚚 Enviado pelo  \n\n8️⃣ **Lâmpada R125 Fisi

In [95]:
search = await extract_data(
  "https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwie-KH3-v2LAxX-LLkGHYTnFG8QFnoECAoQAQ&url=https%3A%2F%2Fwww.techtudo.com.br%2Flistas%2F2024%2F07%2Fmelhor-lampada-inteligente-com-alexa-9-opcoes-para-comprar-pd2024-edinfoeletro.ghtml&usg=AOvVaw3qz9_sSl9VuxNy8uQGR3m3&opi=89978449",
  "1. Lâmpada Inteligente Smart Color 10 W Elgin — a partir de R$ 29 · 2. Lâmpada Inteligente 30 W Elgin — a partir de R$ 72 · 3. Lâmpada Smart ...",
  "Quero uma lampada inteligente"
)
search

[👤] 
    Site: https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwie-KH3-v2LAxX-LLkGHYTnFG8QFnoECAoQAQ&url=https%3A%2F%2Fwww.techtudo.com.br%2Flistas%2F2024%2F07%2Fmelhor-lampada-inteligente-com-alexa-9-opcoes-para-comprar-pd2024-edinfoeletro.ghtml&usg=AOvVaw3qz9_sSl9VuxNy8uQGR3m3&opi=89978449
    Site_info: 1. Lâmpada Inteligente Smart Color 10 W Elgin — a partir de R$ 29 · 2. Lâmpada Inteligente 30 W Elgin — a partir de R$ 72 · 3. Lâmpada Smart ...
    Query: Quero uma lampada inteligente
  
[🤖] Estou na página do Tech Tudo
preciso encontrar as lâmpadas
devo buscar por elementos de lista
-####-
extract_elements('h2', 'True')
[🔍] func: extract_elements args: ['h2', 'True']
[🔍] extracted: Extracted elements:
Element: h2 Classes: header__brand__logo-multicontent Text: AmazonPrime Day
- Element: h2 Classes: content-head__subtitle Text: Aparelhos oferecem opções de agendamento de rotina...
- Element: h2 Text: Lâmpada inteligente com Alexa vale a pena?
- Element: h2 

'\n### Lâmpadas inteligentes encontradas:\n\n1️⃣ **Lâmpada Inteligente Smart Color 10 W Elgin** — a partir de **R$ 29**  \n2️⃣ **Lâmpada Inteligente 30 W Elgin** — a partir de **R$ 72**  \n3️⃣ **Lâmpada Smart Inteligente Intelbras EWS 410** — a partir de **R$ 56**  \n4️⃣ **Smart Lâmpada Wi-Fi RGB+ Positivo** — a partir de **R$ 59**  \n5️⃣ **Lâmpada Wi-Fi inteligente \u200eTP-Link Tapo L530E** — a partir de **R$ 53**  \n6️⃣ **Ekaza Lâmpada Inteligente RGB** — a partir de **R$ 38**'

In [104]:
test = await extract_data(
  "https://www.amazon.com.br",
  "Amazon, site de compras online",
  "Quero roupas de cama tamanho queensize"
)
test

[👤] 
    Site: https://www.amazon.com.br
    Site_info: Amazon, site de compras online
    Query: Quero roupas de cama tamanho queensize
  
[🤖] Estou na Amazon
preciso buscar por roupas de cama
devo achar um input
-####-
extract_elements('input', 'True')
[🔍] func: extract_elements args: ['input', 'True']
[🔍] extracted: Extracted elements:
Element: input Name: field-keywords Placeholder: sem placeholder
- Element: input Classes: nav-bb-button Name: None Placeholder: sem placeholder
- Element: input Classes: a-carousel-firstvisibleitem Name: None Placeholder: sem placeholder
- Element: input Name: ue_back Placeholder: sem placeholder
[🤖] Encontrei o input
preciso buscar por roupas de cama
devo digitar no input
-####-
interact_with_element('input[name="field-keywords"]', 'fill', 'roupas de cama queensize')
[🔍] func: interact_with_element args: ['input[name="field-keywords"]', 'fill', 'roupas de cama queensize']
[🔍] extracted: Texto 'roupas de cama queensize' inserido no elemento 'input[na

'\n### Produtos encontrados:\n\n1️⃣ **Jogo de Lençol 400 Fios Percal Poliéster**  \n⭐ 4.3 (6.434) | **R$47,90** | 2x R$23,95 | 🚚 R$9,35 Frete  \n\n2️⃣ **Jogo de Cama Queen Percal 400 Fios**  \n⭐ 4.5 (1.954) | **R$119,99** | 2x R$60,00 | 🚚 Frete GRÁTIS  \n\n3️⃣ **Jogo de Cama Lençol Queen Bordado**  \n⭐ 4.5 (1.221) | **R$207,87** | 4x R$51,99 | 🚚 Frete GRÁTIS  \n\n4️⃣ **Jogo Roupa de Cama 100% Algodão Teka Crystal**  \n⭐ 4.1 (38) | **R$192,40** | 3x R$64,14 | 🚚 Frete GRÁTIS  \n\n5️⃣ **Jogo de Cama Casal - Lençóis de Luxo**  \n⭐ 4.6 (60) | **R$149,10** | 2x R$74,55 | 🚚 Frete GRÁTIS  \n\n6️⃣ **Kit Lençol Casal Queen 03 Peças**  \n⭐ 5.0 (1) | **R$42,99** | 2x R$21,50 | 🚚 R$9,27 Frete  \n\n7️⃣ **Jogo de Lençol Queen 4 Peças - Hotel de Luxo**  \n⭐ 4.3 (13) | **R$132,90** | 2x R$66,45 | 🚚 Frete GRÁTIS  \n\n8️⃣ **Jogo de Cama Casal Queen Percal 400 Fios**  \n⭐ 3.9 (337) | **R$133,99** | 2x R$67,00 | 🚚 Frete GRÁTIS'